<a href="https://colab.research.google.com/github/tjtyler/MachLearn_MultiLayerPerceptron_BackProp/blob/main/lab_2_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backpropagation Lab





Debug Data = $\begin{bmatrix} -0.4 & 0.3 & 1 \\ -0.3 & 0.8 & 1\\ 
-0.2 & 0.3 & 1\\ -0.1 & 0.9 & 1\\ -0.1 & 0.1 & 0\\ 0.0 & -0.2 & 0\\ 0.1 & 0.2 & 0\\ 0.2 & -0.2 & 0\end{bmatrix}$

$net_{node} = w_{layer} \cdot x$

$z_{node} = \frac{1}{1+e^{net}}$

$\delta_{w_{pq}} = $

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neural_network import MLPClassifier
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. (40%) Correctly implement and submit your own code for the backpropagation algorithm. 

## Code requirements 
- Ability to create a network structure with at least one hidden layer and an arbitrary number of nodes.
- Random weight initialization with small random weights with mean of 0 and a variance of 1.
- Use Stochastic/On-line training updates: Iterate and update weights after each training instance (i.e., do not attempt batch updates)
- Implement a validation set based stopping criterion.
- Shuffle training set at each epoch.
- Option to include a momentum term

Use your Backpropagation algorithm to solve the Debug data. We provide you with several parameters, and you should be able to replicate our results every time. When you are confident it is correct, run your script on the Evaluation data with the same parameters, and print your final weights.

In [3]:
def sigmoid(net):
  """The sigmoid function."""
  return 1.0/(1.0+np.exp(-net))

In [111]:
from os import get_terminal_size
from typing_extensions import ParamSpecKwargs
class MLP(BaseEstimator,ClassifierMixin):

    def __init__(self, arch, epochs, lr=.1, momentum=0, shuffle=True,):
        """ Initialize class with chosen hyperparameters.

        Args:
            arch: a list of nodes at each layer, including the inputs and outputs, but not including bias nodes
            epochs: the number of epochs to run the fit function 
            lr (float): A learning rate / step size.
            shuffle(boolean): Whether to shuffle the training data each epoch. DO NOT SHUFFLE for evaluation / debug datasets.
            momentum(float): The momentum coefficent 
        Example:
            mlp = MLP(lr=.2,momentum=.5,shuffle=False,hidden_layer_widths = [3,3]),  <--- this will create a model with two hidden layers, both 3 nodes wide
        """
        self.arch = arch # [2,2,1] means 2 inputs, 1 hidden layer with 2 nodes, and 1 output
        self.epochs = epochs
        self.lr = lr
        self.momentum = momentum
        self.shuffle = shuffle
        self.layer_wts =[] # list of numpy matrices or vectors with the first element being the weights between the inputs and the first hidden layer, the second element being the weights between the first and second hidden layers, etc.
        self.layer_prev_wts = None
        self.accuracies = []

    def fit(self, X, y, epochs_no_change=11, tol=0.01, default_wts=False, initial_wts=0):
        """ Fit the data; run the algorithm and adjust the weights to find a good solution

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Optional Args (Args we think will make your life easier):
            initial_weights (array-like): allows the user to provide initial weights
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        self.X = X
        self.y = y

        v_sigmoid = np.vectorize(sigmoid) # this converts the sigmoid function to a function that that can take a vector and apply the function to it

        if default_wts:
          self.layer_wts = self.initialize_weights()
        else:
          self.layer_wts = self.initialize_weights_with_value(initial_wts)
        delta_wts = self.layer_wts.copy()
        print("INITIAL WEIGHTS:\n", self.layer_wts)

        #--------------weight initialization working correctly------------------

        self.cur_epoch = 0
        num_epoch_no_change = 0
        last_score = None
        while self.cur_epoch < self.epochs and num_epoch_no_change < epochs_no_change:
          X_bias = None # this will be the input matrix with a column of 1s concatenated for the bias
          if self.shuffle:
            self._shuffle_data(self.X, self.y)
          bias = np.ones((self.X.shape[0],1)) # bias vector of ones
          X_bias = np.concatenate((self.X,bias), axis=1) # set X = X_shuffled with bias concatenated
          #---------------------------------------------------------------------
          for row in range(X_bias.shape[0]):
            Zs_by_layer = []
            nets_by_layer = []
            node_errors_by_layer = []
            delta_wts = self.layer_wts.copy()
            # FORWARD-FEED
            print('\n\nFORWARD-FEED:')
            print('Training Instance data:\n', X_bias[row])
            for lay in range(0, len(self.layer_wts)):
              if lay == 0:
                net = np.matmul(X_bias[row], self.layer_wts[lay])
                nets_by_layer.append(net)
                Zs_lay = v_sigmoid(net)
                Zs_by_layer.append(Zs_lay)
              else:
                Zs_by_layer[lay-1] = np.insert(Zs_by_layer[lay-1],Zs_by_layer[lay-1].shape[0],[1]) # add a one to the Zs_by_layer for the bias
                net = np.matmul(Zs_by_layer[lay-1], self.layer_wts[lay])
                nets_by_layer.append(net)
                Zs_lay = v_sigmoid(net)
                Zs_by_layer.append(Zs_lay) 
                print(f'Zs:\n', Zs_by_layer) 

            # BACK PROPAGATE
            print('\nBACK-PROPAGATE:')
            target_vec = self.get_target_vec(row)
            target_vec = target_vec.flatten()
            for lay_indx in reversed(range(0, len(self.layer_wts))):
              i = 0 # need this to access the node_erros from previous layer to calculate next layer (working backwards)
              if lay_indx == (len(self.layer_wts) - 1): 
                print('COMPUTING ERROR AT OUTPUT NODES')
                # COMPUTE THE ERROR AT THE OUTPUT NODES
                  # (target_j - z_j)*z_j*(1 - z_j)
                OutNodes_error = (target_vec - Zs_by_layer[lay_indx])*Zs_by_layer[lay_indx]*(np.ones(Zs_by_layer[lay_indx].shape) - Zs_by_layer[lay_indx])
                # ADD THE ERROR AT OUTPUT NODES VECTOR TO node_errors_by_layer
                node_errors_by_layer.append(OutNodes_error)

              else:
                # CALCULATE CHANGE IN WEIGHTS
                print(f'CHANGE IN WEIGHTS BETWEEN LAYER {lay_indx+1} AND LAYER {lay_indx+ 2}')
                  # delta_wts_ij = learning_rate * error_at_node_j * z_i
                lr_vec_shape = node_errors_by_layer[i-1].shape
                lr_vec = np.full(lr_vec_shape, self.lr) 
                Zs = Zs_by_layer[lay_indx]
                Zs = Zs.reshape(Zs.shape[0],1) # make Zs a column vector
                errors_at_nodes = node_errors_by_layer[i-1]
                delta_wts[lay_indx+1] = Zs * lr_vec * errors_at_nodes 

                # COMPUTE THE ERROR AT HIDDEN LAYERS
                print('COMPUTING ERROR AT HIDDEN LAYER NODES')
                  # error_node_j = Sum(error_node_k * wt_j->k)*f_prime_net_j
                delta_dot_w = np.matmul(self.layer_wts[lay_indx+1],node_errors_by_layer[i-1])
                delta_dot_w = delta_dot_w[:-1] # exclude the bias node
                  # f_prime_net = z_j*(1 - z_j)
                f_prime_net = (Zs_by_layer[lay_indx]) * (np.ones( (Zs_by_layer[lay_indx].shape[0]) ) - Zs_by_layer[lay_indx]) 
                f_prime_net = f_prime_net[:-1] # exclude the bias node
                # ADD THE ERROR AT HIDDEN LAYER NODES VECTOR TO node_errors_by_layer
                node_errors_by_layer.append(np.multiply(delta_dot_w ,f_prime_net))

                if lay_indx == 0:
                  lr_vec_shape = node_errors_by_layer[-1].shape
                  lr_vec = np.full(lr_vec_shape, self.lr) 
                  inputs = X_bias[row]
                  inputs = inputs.reshape(inputs.shape[0],1) # make inputs a column vector
                  errors_at_nodes = node_errors_by_layer[-1]
                  delta_wts[lay_indx] = inputs * lr_vec * errors_at_nodes       
              i += 1

            node_errors_by_layer.reverse()
            print('PREVIOUS WEIGHTS\n', self.layer_wts) 
            for i in range(len(self.layer_wts)):
              self.layer_wts[i] = self.layer_wts[i] + delta_wts[i]
            print('NODE ERRORS BY LAYER\n',node_errors_by_layer)
            print('CHANGE IN WEIGHTS\n', delta_wts)
            print('NEW UPDATED WEIGHTS\n', self.layer_wts)
          #---------------

          # if self.cur_epoch == 0:
          #   last_score = self.score(self.X,self.y)
          #   self.accuracies.append(last_score)
          # elif self.cur_epoch > 0:
          #   cur_score = self.score(self.X,self.y)
          #   if ((cur_score - last_score)**2)**(1/2) < tol:
          #     num_epoch_no_change +=1
          #   else:
          #     num_epoch_no_change = 0
          #   last_score = cur_score
          #   self.accuracies.append(last_score)
          
          self.cur_epoch +=1
        
        return self

    def get_target_vec(self,row):
      # THIS RETURNS A COLUMN VECTOR
      target_value = int(self.y[row][0])
      target_vector = np.zeros(shape=(self.arch[-1],1))
      target_vector[target_value][0] = 1
      return target_vector
      

    def predict(self, X):
        """ 
            Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding 
            targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        # bias = np.ones((X.shape[0],1)) # bias vector of ones
        # X_bias = np.concatenate((X,bias), axis=1) # set X = X_shuffled with bias concatenated
        # predictions = np.zeros([X.shape[0],1])
        # for row in range(0, X.shape[0]):
        #   net = self.calcNet(X_bias, row)
        #   output = self.output(net)
        #   predictions[row][0] = output
        # return predictions
        return 0

    def initialize_weights_with_value(self, val):
      # if self.arch = [2,3,2] then the lay1_wts have a shape of (3,3) and lay2_wts a shape of (4,2)
        for i in range(0, len(self.arch) - 1):
          self.layer_wts.append(np.full((self.arch[i] + 1, self.arch[i+1]),val))
        return self.layer_wts

    def initialize_weights(self):
        """ Initialize weights for perceptron. Don't forget the bias!
        Returns:
        """
        # if self.arch = [2,3,2] then the lay1_wts have a shape of (3,3) and lay2_wts a shape of (4,2)
        for i in range(0, len(self.arch) - 1):
          self.layer_wts.append(np.random.randn(self.arch[i] + 1, self.arch[i+1]))
        return self.layer_wts

    # def initialize_zero_weights(self):
    #     """ Initialize weights for perceptron. Don't forget the bias!

    #     Returns:

    #     """
    #     # if self.arch = [2,3,2] then the lay1_wts have a shape of (3,3) and lay2_wts a shape of (4,2)
    #     for i in range(0, len(self.arch) - 1):
    #       self.layer_wts.append(np.zeros((self.arch[i] + 1, self.arch[i+1])))
    #     return self.layer_wts

    def score(self, X, y):
        """ 
            Return accuracy of model on a given dataset. Must implement own 
            score function.
        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets
        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """
        self._shuffle_data(X,y)
        predictions = self.predict(X)
        correct = 0
        total = y.shape[0]
        for i in range(0, y.shape[0]):
          if predictions[i][0] == y[i][0]:
            correct += 1
        return correct/total


    def _shuffle_data(self, X, y):
        """ Shuffle the data! This _ prefix suggests that this method should only be called internally.
            It might be easier to concatenate X & y and shuffle a single 2D array, rather than
             shuffling X and y exactly the same way, independently.
        """
        single_arr = np.concatenate((X,y), axis=1) # concatenate X and y into a single array
        np.random.shuffle(single_arr) # shuffle the rows of the concatenated X-y array
        cutoff = single_arr.shape[1] - 1 # the point to split the X and y arrays after shuffling
        X = single_arr[:,:cutoff] # the shuffled X array
        y = single_arr[:,cutoff:] # the shuffled y array

    ### Not required by sk-learn but required by us for grading. Returns the weights.
    def get_weights(self):
        return self.init_layer_wts


## 1.1 

Debug your model using the following parameters:

Learning Rate = 0.1\
Momentum = 0.5\
Deterministic = 10 [This means run it 10 epochs and should be the same everytime you run it]\
Shuffle = False\
Validation size = 0\
Initial Weights = All zeros\
Hidden Layer Widths = [4]

---

### 1.1.1 Debug 

Debug your model by running it on the [Debug Dataset](https://byu.instructure.com/courses/14142/files?preview=4421290)


Expected Results for Binary Classification (i.e. 1 output node): [debug_bp_0.csv](https://byu.instructure.com/courses/14142/files?preview=4537323) 

$$ \text{Layer 1} = \begin{bmatrix} -8.81779797\text{e}-05 & -8.81779797\text{e}-05 & -8.81779797\text{e}-05 & -8.81779797\text{e}-05 \\ 7.82757731\text{e}-04 & 7.82757731\text{e}-04 & 7.82757731\text{e}-04 & 7.82757731\text{e}-04 \\ -3.94353645\text{e}-03 & -3.94353645\text{e}-03 & -3.94353645\text{e}-03 & -3.94353645\text{e}-03 \end{bmatrix}$$
                                             
$$ \text{Layer 2} = \begin{bmatrix} -0.01060888 \\ -0.01060888 \\ -0.01060888 \\ -0.01060888 \\ -0.02145495 \end{bmatrix}$$

(The weights do not need to be in this order or shape.)

Expected Results for One Hot Vector Classification (i.e. 2 output nodes): [debug_bp_2outs.csv](https://byu.instructure.com/courses/14142/files?preview=4537340) 

$$ \text{Layer 1} = \begin{bmatrix} -0.00018149 & -0.00018149 & -0.00018149 & -0.00018149 \\ 0.00157468 & 0.00157468 & 0.00157468 & 0.00157468 \\ -0.00788218 & -0.00788218 & -0.00788218 & -0.00788218 \end{bmatrix}$$
                          
$$ \text{Layer 2} = \begin{bmatrix} 0.01050642 & -0.01050642 \\ 0.01050642 & -0.01050642 \\ 0.01050642 & -0.01050642 \\ 0.01050642 & -0.01050642 \\ 0.02148778 & -0.02148778 \end{bmatrix}$$

(The weights do not need to be in this order or shape.)

In [112]:
from scipy.io.arff import loadarff 
import pandas as pd
# Load debug data

# Train on debug data

# Print weights


# LOAD DEBUG DATA
raw_data = loadarff('/content/drive/MyDrive/School/CS_472_MachLearning/labs/lab2_backpropagation/data/linsep2nonorigin.arff')
df_data = pd.DataFrame(raw_data[0])

# CHANGE THE VALUES OF THE ROWS OF THE 'class' COLUMN TO BE 1s AND 0s INSTEAD OF "b'1'" OR "b'0'"
df_data['class'] = df_data['class'].astype(int) 

np_arr = df_data.to_numpy() #cast dataframe to numpy array
# np_arr = np.array([[-1,0.4,0.2]])
# np_arr = np.array([[.9,.6,0]])
cutoff = np_arr.shape[1] - 1 #the index to split the X and y arrays

# CUT THE numpy array INTO AN X AND y ARRAY
X = np_arr[:,:cutoff] #inputs: from 1st column to cutoff (exclusive)
y_2D = np_arr[:,cutoff:] #targets: from cutoff to last column

# SET INITIAL WEIGHTS TO ZERO
wts = np.zeros((X.shape[1]+1,1)) # weights

# TRAIN ON DEBUG DATA
mlp = MLP([2,4,2],2,0.1,0)
mlp.fit(X,y_2D,epochs_no_change=11,tol=0.01,default_wts=False,initial_wts=0)



# PRINT ACCURACY AND WEIGHTS

INITIAL WEIGHTS:
 [array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])]


FORWARD-FEED:
Training Instance data:
 [-0.4  0.3  1. ]
Zs:
 [array([0.5, 0.5, 0.5, 0.5, 1. ]), array([0.5, 0.5])]

BACK-PROPAGATE:
COMPUTING ERROR AT OUTPUT NODES
CHANGE IN WEIGHTS BETWEEN LAYER 1 AND LAYER 2
COMPUTING ERROR AT HIDDEN LAYER NODES
PREVIOUS WEIGHTS
 [array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])]
NODE ERRORS BY LAYER
 [array([0., 0., 0., 0.]), array([-0.125,  0.125])]
CHANGE IN WEIGHTS
 [array([[-0., -0., -0., -0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]]), array([[-0.00625,  0.00625],
       [-0.00625,  0.00625],
       [-0.00625,  0.00625],
       [-0.00625,  0.00625],
       [-0.0125 ,  0.0125 ]])]
NEW UPDATED WEIGHTS
 [array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), 

MLP(arch=[2, 4, 2], epochs=2)

### 1.1.2 Evaluation

Evaluate your model using the following parameters:

Learning Rate = 0.1\
Momentum = 0.5\
Deterministic = 10 [This means run it 10 epochs and should be the same everytime you run it]\
Shuffle = False\
Validation size = 0\
Initial Weights = All zeros\
Hidden Layer Widths = [4]

We will evaluate your model based on printed weights after training on the [Evaluation Dataset](https://byu.instructure.com/courses/14142/files?preview=4421294)

In [ ]:
# Load evaluation data

# Train on evaluation data

# Print weights

## 2. (10%) Backpropagation on the Iris Classification problem.

Load the Iris Dataset [Iris Dataset](https://byu.instructure.com/courses/14142/files?preview=4421369)

Parameters:
- One layer of hidden nodes with the number of hidden nodes being twice the number of inputs.
- Use a 80/20 split of the data for the training/test set.
- Use a learning rate of 0.1
- Use a validation set (15% of the training set) taken from the training set for your stopping criteria
- Create one graph with MSE (mean squared error) over epochs from the training set and validation set
- Create one graph with classification accuracy (% classified correctly) over epochs from the training set and validation set
- Print out your test set accuracy

The results for the different measurables should be shown with a different color, line type, etc. Typical backpropagation accuracies for the Iris data set are 85-95%.

---

In [ ]:
# Iris Classification

## 3. (10%) Working with the Vowel Dataset - Learning Rate

Load the Vowel Dataset [Vowel Dataset](https://byu.instructure.com/courses/14142/files?preview=4537354)

- Use one layer of hidden nodes with the number of hidden nodes being twice the number of inputs.
- Use random 80/20 splits of the data for the training/test set.
- Use a validation set (15% of the training set) taken from the training set for your stopping criteria
- Try some different learning rates (LR). Note that each LR will probably require a different number of epochs to learn. 

- For each LR you test, plot their validation's set MSE over Epochs on the same graph. Graph 4-5 different LRs and make them different enough to see a difference between them.

In general, whenever you are testing a parameter such as LR, # of hidden nodes, etc., test values until no more improvement is found. For example, if 20 hidden nodes did better than 10, you would not stop at 20, but would try 40, etc., until you no longer get improvement.

If you would like you may average the results of multiple initial conditions (e.g. 3) per LR, and that obviously would give more accurate results.

<img src=https://raw.githubusercontent.com/cs472ta/CS472/master/images/backpropagation/backprop_val_set_MSE_vs_epochs.png width=500 height=500  align="left">

In [ ]:
# Train on each dataset

## 3.1 (5%) Working with the Vowel Dataset - Intuition
- Discuss the effect of varying learning rates. 
- Discuss why the vowel data set might be more difficult than Iris
    - Report both datasets' baseline accuracies and best **test** set accuracies. 
- Consider which of the vowel dataset's given input features you should actually use (Train/test, speaker, gender, ect) and discuss why you chose the ones you did.

Typical backpropagation accuracies for the Vowel data set are above 75%.



*Discuss Intuition here*



## 3.2 (10%) Working with the Vowel Dataset - Hidden Layer Nodes

Using the best LR you discovered, experiment with different numbers of hidden nodes.

- Start with 1 hidden node, then 2, and then double them for each test until you get no more improvement in accuracy. 
- For each number of hidden nodes find the best validation set solution (in terms of validation set MSE).  
- Create one graph with MSE for the training set and validation set on the y-axis and # of hidden nodes on the x-axis.
- Report the final test set accuracy for every # of hidden nodes you experimented on

*Discuss Hidden Layer Nodes here*



## 3.3 (10%) Working with the Vowel Dataset - Momentum

Try some different momentum terms using the best number of hidden nodes and LR from your earlier experiments.

- Create a graph similar to step 3.2, but with momentum on the x-axis and number of epochs until validation set convergence on the y-axis.
- For each momentum term, print the test set accuracy. 
- You are trying to see how much momentum speeds up learning and how it affects accuracy.

*Discuss Momentum here*



## 4.1 (10%) Use the scikit-learn (SK) version of the MLP classifier on the Iris and Vowel data sets.  

You do not need to go through all the steps above, nor graph results. Compare results (accuracy and learning speed) between your version and theirs for some selection of hyper-parameters. Try different hyper-parameters and comment on their effect.

At a minimum, try

- number of hidden nodes and layers
- different activation functions
- learning rate
- regularization and parameters
- momentum (and try nesterov)
- early stopping

In [ ]:
# Load sklearn perceptron

# Train on voting dataset

*Record impressions*

## 4.2 (5%) Using the Iris Dataset automatically adjust hyper-parameters using your choice of grid/random search
- Use a grid or random search approach across a reasonable subset of hyper-parameters from the above 
- Report your best accuracy and hyper-parameters. 

In [ ]:
# Load sklearn perceptron

# Train on voting dataset

## 5. (Optional 5% Extra credit) For the vowel data set, use the other hyper-parameter approach that you did not use in part 4.2 to find LR, # of hidden nodes, and momentum.  

- Compare and discuss the values found with the ones you found in part 3.


*Discuss findings here*